Story Generation using LSTM Model

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_stories = pd.read_csv("story_generation_dataset/ROCStories_train.csv", encoding="utf8")
test_stories = pd.read_csv("story_generation_dataset/ROCStories_test.csv", encoding="utf8")
val_stories = pd.read_csv("story_generation_dataset/ROCStories_val.csv", encoding="utf8")

In [3]:
train_stories = train_stories.append(val_stories)
train_stories = train_stories[:2000]
test_stories = train_stories[:500]

In [4]:
train_stories.head()

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet..."
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for...
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...
4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,His congregation was delighted and so was he.


In [5]:
test_stories.head()

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet..."
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for...
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...
4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,His congregation was delighted and so was he.


In [6]:
train_array = train_stories.values[:12000,1:].reshape(-1).tolist()
test_array = test_stories.values[:,1:].reshape(-1).tolist()

In [7]:
vecn = np.load("data/train_vectors.npy")
vec = vecn.tolist()
print(len(vec))

12000


In [8]:
def get_nearest(qvec, vectors, array, k=5):
      sentences = []
      qvec /= np.linalg.norm(qvec)
      vectors /= np.linalg.norm(vectors)
      scores = np.dot(qvec, vectors.T).flatten()
      sorted_args = np.argsort(scores)[::-1]
      for i in range(k):
            sentences.append(array[sorted_args[i]])
      for i, s in enumerate(sentences):
            print (sorted_args[i],'\t',s)

In [9]:
vt = vec[::6]
v1 = vec[1::6]
v2 = vec[2::6]
v3 = vec[3::6]
v4 = vec[4::6]
v5 = vec[5::6]

In [10]:
len(vt),len(v1),len(v2),len(v3),len(v4)

(2000, 2000, 2000, 2000, 2000)

In [11]:
v0 = np.zeros(2400).tolist()
X = []
y = []
m = len(v1)
for i in range(m):
    X.append([v0 , v0 , v0 , v1[i]])
    y.append(v2[i])
    X.append([v0, v0, v1[i], v2[i]])
    y.append(v3[i])
    X.append([v0, v1[i], v2[i], v3[i]])
    y.append(v4[i])
    X.append([v1[i], v2[i], v3[i], v4[i]])
    y.append(v5[i])
    
X = np.asarray(X)   # X.shape = (samples, timesteps, features)
y = np.asarray(y)
print(X.shape)

(8000, 4, 2400)


In [12]:
maxvec = 10
for i in vec:
    if min(i) < maxvec:
        maxvec = min(i)
maxvec

-0.18974100053310394

In [13]:
from keras.preprocessing import sequence 
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Embedding, LSTM 
model = Sequential() 
model.add(LSTM(2400, dropout=0.2, recurrent_dropout=0.2, input_shape=(4,2400))) 
model.compile(loss='mean_squared_error', optimizer='rmsprop') 

In [14]:
model.fit(X, y, batch_size=64, epochs=10)

Epoch 1/10
125/125 [==============================] - 263s 2s/step - loss: 2.8260e-04
Epoch 2/10
125/125 [==============================] - 280s 2s/step - loss: 2.6583e-04
Epoch 3/10
125/125 [==============================] - 279s 2s/step - loss: 2.6105e-04
Epoch 4/10
125/125 [==============================] - 280s 2s/step - loss: 2.5769e-04
Epoch 5/10
125/125 [==============================] - 272s 2s/step - loss: 2.5516e-04
Epoch 6/10
125/125 [==============================] - 268s 2s/step - loss: 2.5284e-04
Epoch 7/10
125/125 [==============================] - 297s 2s/step - loss: 2.5103e-04
Epoch 8/10
125/125 [==============================] - 298s 2s/step - loss: 2.4941e-04
Epoch 9/10
125/125 [==============================] - 296s 2s/step - loss: 2.4808e-04
Epoch 10/10
125/125 [==============================] - 265s 2s/step - loss: 2.4652e-04


In [15]:
from random import randint
test_index = []
for i in range(20):
    test_index.append(randint(0,6000))
np.savetxt("test-index.txt",test_index,fmt="%s")

In [16]:
test_ls = np.loadtxt("test-index.txt")
sim_ls = []
for i in range(20):
    j = int(test_ls[i])
    print("Original Story:")
    get_nearest(X[j:j+1,3,:].squeeze().tolist(), vecn, train_array, k=1)
    print("Actual Story:")
    get_nearest(y[j:j+1,:].squeeze().tolist(), vecn, train_array, k=1)
    print("Predicted Story:")
    pred = model.predict(X[j:j+1,:,:])
    get_nearest(pred.squeeze().tolist(), vecn, train_array)
    sim = np.dot(pred.squeeze(), y[j:j+1,:].squeeze().T)
    print("Similarity:",sim,end="\n\n")
    #Should be close to 1, that means they are same (1 when normalized)
    sim_ls.append(sim)

Original Story:
2032 	 We had no reservations and had to sit at the bar.
Actual Story:
2033 	 We had a great time catching up, and vowed to meet soon.
Predicted Story:
2888 	 We went to the theater and purchased the tickets.
749 	 After two weeks the family returned home excited for the next vacation.
2891 	 We went back to the entrance and was able to see the correct movie.
9573 	 We went to the theater and bought popcorn and drinks.
311 	 They went to the concert and had a great time.
Similarity: 0.4037988943634146

Original Story:
2222 	 I dreamed my friend's daughter broke up with her boyfriend.
Actual Story:
2223 	 As a result her grades in college were falling.
Predicted Story:
4933 	 When I came home from school, I dressed up for a party.
3697 	 I decided that I wanted to see one new movie each week.
1831 	 When I was young I went to a slumber party at my friend Tim's house.
8944 	 I looked for a job and home the entire time I stayed there.
4417 	 I was visiting my hometown for 

In [17]:
print('Average Similarity =',np.average(sim_ls))

Average Similarity = 0.37811440144545105
